In [2]:
#!/usr/bin/env python3
"""
Requirements:
    pip install yt-dlp ffmpeg-python opencv-python
"""
# ⚙️ Imports
import os
from pathlib import Path
import cv2
import ffmpeg                  # pip install ffmpeg-python
import yt_dlp                   # pip install -U yt-dlp

import argparse
from yt_dlp import YoutubeDL
from typing import Optional, Tuple


In [10]:


# 📥 1) Download --------------------------------------------------------------
def download_video(
    youtube_url: str,
    video_path: str = "video.mp4",
    cookies_from: Optional[Tuple[str, ...]] = ("chrome",),
    cookiefile: Optional[str] = None,
) -> str:
    ydl_opts = {
        "format": "bestvideo+bestaudio/best",
        "outtmpl": f"{Path(video_path).with_suffix('')}.%(ext)s",
        "merge_output_format": "mp4",
        "noplaylist": True,
        "quiet": True,
    }
    if cookies_from:
        ydl_opts["cookiesfrombrowser"] = cookies_from
    if cookiefile:
        ydl_opts["cookiefile"] = cookiefile

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        video_file = ydl.prepare_filename(info)

    print(f"✅ Downloaded → {video_file}")
    return video_file


# 📸 2-A) Extract with FFmpeg -------------------------------------------------
def extract_frames_ffmpeg(video_path: str,
                          out_dir: str = "frames",
                          fps: float = 1.0):
    os.makedirs(out_dir, exist_ok=True)
    (ffmpeg
        .input(video_path)
        .filter("fps", fps=fps)
        .output(f"{out_dir}/%06d.jpg", vsync="vfr")
        .run(quiet=True, overwrite_output=True)
    )
    print(f"✅ Extracted frames at {fps} FPS → {out_dir}")


# 📸 2-B) Extract with OpenCV -------------------------------------------------
def extract_frames_cv(video_path: str,
                      out_dir: str = "frames",
                      every_sec: float = 30.0):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    step = int(fps * every_sec)
    count = saved = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % step == 0:
            cv2.imwrite(f"{out_dir}/{saved:06d}.jpg", frame)
            saved += 1
        count += 1

    cap.release()
    print(f"✅ Saved {saved} frames (1 every {every_sec}s) → {out_dir}")


# 🚀 3) Run the pipeline ------------------------------------------------------
YOUTUBE_URL = "https://youtu.be/QePQ2sJJ5DA"   # private video
VIDEO_FILE  = "yagami_video.mp4"
FRAMES_DIR  = "frames_yagami_face_bday"
FPS         = .1    # e.g. 2.0 to use ffmpeg
INTERVAL_S  = 10.0    # e.g. 10-second spacing (OpenCV path)

video_path = download_video(YOUTUBE_URL, VIDEO_FILE,
                            cookies_from=None,
                            cookiefile='/Users/christopherclark/Library/Mobile Documents/com~apple~CloudDocs/_Chris_Docs/Coding/darksharkml/studio.youtube.com_cookies.txt')  # or cookiefile="cookies.txt"

if FPS is not None:
    extract_frames_ffmpeg(video_path, FRAMES_DIR, fps=FPS)
else:
    extract_frames_cv(video_path, FRAMES_DIR, every_sec=INTERVAL_S)

print("\n🎬 All done! Drag-drop the ‘frames_*’ folder into Roboflow Annotate.")


✅ Downloaded → yagami_video.mp4
✅ Extracted frames at 0.1 FPS → frames_yagami_face_bday

🎬 All done! Drag-drop the ‘frames_*’ folder into Roboflow Annotate.


In [2]:
from roboflow import Roboflow
import os

# Initialize the Roboflow object with your API key
rf = Roboflow(api_key="3Tq8ARsAlvjXztKdXHgy")

# Retrieve your current workspace and project name
print(rf.workspace())

# Specify the project for upload
# let's you have a project at https://app.roboflow.com/my-workspace/my-project
workspaceId = 'darksharkai'
projectId = 'darksharkai-81gsv'
project = rf.workspace(workspaceId).project(projectId)

# Upload the image to your project
img_directory = "/Users/christopherclark/Library/Mobile Documents/com~apple~CloudDocs/_Chris_Docs/Coding/darksharkml/frames_yagami_face_bday"

for filename in os.listdir(img_directory):
    # build the full path
    filepath = os.path.join(img_directory, filename)
    # optionally skip non-files or filter by extension
    if os.path.isfile(filepath) and filename.lower().endswith((".jpg", ".jpeg", ".png")):
        project.upload(filepath)
        # print(f"Uploaded {filename}")

"""
Optional Parameters:
- num_retry_uploads: Number of retries for uploading the image in case of failure.
- batch_name: Upload the image to a specific batch.
- split: Upload the image to a specific split.
- tag: Store metadata as a tag on the image.
- sequence_number: [Optional] If you want to keep the order of your images in the dataset, pass sequence_number and sequence_size..
- sequence_size: [Optional] The total number of images in the sequence. Defaults to 100,000 if not set.
"""

project.upload(
    image_path="UPLOAD_IMAGE.jpg",
    batch_name="YOUR_BATCH_NAME",
    split="train",
    num_retry_uploads=3,
    tag_names=["YOUR_TAG_NAME"],
    sequence_number=99,
    sequence_size=100
)


loading Roboflow workspace...
{
  "name": "darksharkai",
  "url": "darksharkai",
  "projects": [
    "darksharkai/darksharkai-81gsv"
  ]
}
loading Roboflow workspace...
loading Roboflow project...


RuntimeError: The provided image path [ UPLOAD_IMAGE.jpg ] is not a valid path. Please provide a path to an image or a directory.